In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers, regularizers, optimizers

In [ ]:
import pickle
(x_train, y_train), (x_validate, y_validate) = \
    pickle.load(open("dataset_encoded.p", "rb"))
print(x_train.shape)
print(y_train.shape)
print(x_validate.shape)
print(y_validate.shape)
import tensorflow as tf
split_index = int(0.8 * len(x_train))
dataset_train = tf.data.Dataset.from_tensor_slices(
    (x_train[:split_index], y_train[:split_index])
).cache()
dataset_validate = tf.data.Dataset.from_tensor_slices(
    (x_train[split_index:], y_train[split_index:])
).cache()

dataset_test =tf.data.Dataset.from_tensor_slices((x_validate, y_validate))

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)
x_train[48].shape

In [ ]:
y_train.sum(axis=0)

In [ ]:
(data_x, data_y), (data_validate, data_validate_y) =pickle.load(open("dataset_encoded.p", "rb"))

In [ ]:
data = np.concatenate((data_y[0], data_y[1]), axis=1) 

In [ ]:
dataset_train

In [ ]:
model = models.Sequential()
model.add(layers.Flatten(input_shape=(64,80,1)))
#model.add(layers.LSTM(320 , input_shape=(64,80)))
#model.add(layers.Dense(320 , activation='tanh', kernel_regularizer=regularizers.l2(0.01)))
#model.add(layers.Dense(120 , activation='relu', kernel_regularizer=regularizers.l2(0.01)))
#model.add(layers.Dense(80, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))
#model.add(layers.Dense(320 , activation='tanh', kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dense(120 , activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dense(80, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))
model.summary()

model.compile(optimizer=optimizers.Nadam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
history = model.fit(
    dataset_train.shuffle(10000).batch(32), # pipeline; batch - too high - memory will explode and overfitting; too low - slow
    epochs=10,
    validation_data=dataset_validate.batch(32))

In [ ]:
plt.plot(history.history["loss"], label=['loss'])
plt.plot(history.history["val_loss"], label=['val_loss'])
plt.legend()
plt.show()
plt.close()


plt.plot(history.history["accuracy"], label=['accuracy'])
plt.plot(history.history["val_accuracy"], label=['val_accuracy'])
plt.legend()
plt.show()
plt.close()

In [ ]:
import random
import numpy as np
def generate(model, seed, temperature, generated_sequence_length):
    
    input_sequence = list(seed)
    # Generate the sequence by repeatedly predicting.
    generated_sequence = []
    while len(generated_sequence) < generated_sequence_length:
        prediction = model.predict(np.array([seed]))
        predicted_index = get_index_from_prediction(prediction[0], temperature)
        generated_sequence.append(predicted_index)
        input_sequence = input_sequence[1:]
        input_sequence.append(prediction[0])
    # Convert the generated sequence to a string.
    return [np.argmax(x) for x in seed] + list(generated_sequence)

def get_index_from_prediction(prediction, temperature=0.0):
    """ Gets an index from a prediction. """
    # Zero temperature - use the argmax.
    if temperature == 0.0:
        return np.argmax(prediction)
    # Non-zero temperature - do some random magic.
    else:
        prediction = np.asarray(prediction).astype('float64')
        prediction = np.log(prediction) / temperature
        exp_prediction= np.exp(prediction)
        prediction = exp_prediction / np.sum(exp_prediction)
        probabilities = np.random.multinomial(1, prediction, 1)
        return np.argmax(probabilities)
random_index = random.randint(0, len(x_train))
random_index = 0
seed = x_train[random_index]
generated_sequence = generate(model, seed=seed, temperature=1.0, generated_sequence_length=200)
plt.plot(generated_sequence)
plt.show()
plt.close()

In [ ]:
import note_seq
def play(sequence):
    note_sequence = note_seq.protobuf.music_pb2.NoteSequence()
    note_sequence.tempos.add().qpm = 120.0
    note_sequence.ticks_per_quarter = note_seq.constants.STANDARD_PPQ
    note_sequence.total_time = 0.0
    time = 0.0
    sixteenth_length = 2.0 / 16.0
    pitch = None
    start_time = None
    for n in sequence:
        if n < 4 and pitch is None:
            continue
        elif n < 4 and pitch != None:
            note = note_seq.protobuf.music_pb2.NoteSequence.Note()
            note.pitch = pitch + 24
            note.velocity = 100
            note.start_time = start_time
            note.end_time = time
            note_sequence.notes.append(note)
            pitch = None
            start_time = None
        elif n >= 4 and pitch != None:
            note = note_seq.protobuf.music_pb2.NoteSequence.Note()
            note.pitch = pitch + 24
            note.velocity = 100
            note.start_time = start_time
            note.end_time = time
            note_sequence.notes.append(note)
            pitch = n
            start_time = time
        elif n >= 4 and pitch is None:
            pitch = n
            start_time = time
        else:
            assert False
        time += sixteenth_length
    note_seq.plot_sequence(note_sequence)
    note_seq.play_sequence(note_sequence, synth=note_seq.fluidsynth)
play(generated_sequence)